In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Datacamp Pyspark Tutorial")\
.config("spark.memory.offHeap. enabled", "true") . config("spark.memory.offHeap. size", "10g") \
.getOrCreate()


In [8]:
# Define the file path
file_path = r"C:\Users\abdel\OneDrive\Desktop\demographic.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first 30 rows of the DataFrame
df.show(5,0)
df.printSchema()
df.count()

+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+-------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+-------+--------+--------+--------+--------+
|SEQN |SDDSRVYR|RIDSTATR|RIAGENDR|RIDAGEYR|RIDAGEMN|RIDRETH1|RIDRETH3|RIDEXMON|RIDEXAGM|DMQMILIZ|DMQADFC|DMDBORN4|DMDCITZN|DMDYRSUS|DMDEDUC3|DMDEDUC2|DMDMARTL|RIDEXPRG|SIALANG|SIAPROXY|SIAINTRP|FIALANG|FIAPROXY|FIAINTRP|MIALANG|MIAPROXY|MIAINTRP|AIALANGA|DMDHHSIZ|DMDFMSIZ|DMDHHSZA|DMDHHSZB|DMDHHSZE|DMDHRGND|DMDHRAGE|DMDHRBR4|DMDHREDU|DMDHRMAR|DMDHSEDU|WTINT2YR    |WTMEC2YR    |SDMVPSU|SDMVSTRA|INDHHIN2|INDFMIN2|INDFMPIR|
+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+

10175

In [10]:

df= df.dropna()
df.show()

+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+-------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+
|SEQN|SDDSRVYR|RIDSTATR|RIAGENDR|RIDAGEYR|RIDAGEMN|RIDRETH1|RIDRETH3|RIDEXMON|RIDEXAGM|DMQMILIZ|DMQADFC|DMDBORN4|DMDCITZN|DMDYRSUS|DMDEDUC3|DMDEDUC2|DMDMARTL|RIDEXPRG|SIALANG|SIAPROXY|SIAINTRP|FIALANG|FIAPROXY|FIAINTRP|MIALANG|MIAPROXY|MIAINTRP|AIALANGA|DMDHHSIZ|DMDFMSIZ|DMDHHSZA|DMDHHSZB|DMDHHSZE|DMDHRGND|DMDHRAGE|DMDHRBR4|DMDHREDU|DMDHRMAR|DMDHSEDU|WTINT2YR|WTMEC2YR|SDMVPSU|SDMVSTRA|INDHHIN2|INDFMIN2|INDFMPIR|
+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-

In [17]:
# Assuming 'Titanic_df' is your PySpark DataFrame

# Import necessary functions from PySpark
from pyspark.sql.functions import col, sum as spark_sum

# Calculate the count of null values in each column
null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Convert the result to a dictionary format for easier analysis
null_counts_dict = null_counts.first().asDict()

# Print the count of null values in each column
for column, count in null_counts_dict.items():
    print(f"Column '{column}': {count} null values")

Column 'SEQN': 0 null values
Column 'SDDSRVYR': 0 null values
Column 'RIDSTATR': 0 null values
Column 'RIAGENDR': 0 null values
Column 'RIDAGEYR': 0 null values
Column 'RIDAGEMN': 9502 null values
Column 'RIDRETH1': 0 null values
Column 'RIDRETH3': 0 null values
Column 'RIDEXMON': 362 null values
Column 'RIDEXAGM': 5962 null values
Column 'DMQMILIZ': 3914 null values
Column 'DMQADFC': 9632 null values
Column 'DMDBORN4': 0 null values
Column 'DMDCITZN': 4 null values
Column 'DMDYRSUS': 8267 null values
Column 'DMDEDUC3': 7372 null values
Column 'DMDEDUC2': 4406 null values
Column 'DMDMARTL': 4406 null values
Column 'RIDEXPRG': 8866 null values
Column 'SIALANG': 0 null values
Column 'SIAPROXY': 1 null values
Column 'SIAINTRP': 0 null values
Column 'FIALANG': 121 null values
Column 'FIAPROXY': 121 null values
Column 'FIAINTRP': 121 null values
Column 'MIALANG': 2864 null values
Column 'MIAPROXY': 2863 null values
Column 'MIAINTRP': 2862 null values
Column 'AIALANGA': 3858 null values
Colu

In [20]:
from pyspark.sql.functions import col, median

# Calculate the median value for the 'RIDAGEMN' column
median_RIDAGEMN = df.select(median('RIDAGEMN')).collect()[0][0]

# Fill null values in the 'RIDAGEMN' column with the median value
df = df.withColumn('RIDAGEMN', 
                   when(col('RIDAGEMN').isNull(), median_RIDAGEMN).otherwise(col('RIDAGEMN')))


In [29]:
from pyspark.sql.functions import col, when, median
# Identify numeric columns with null values
numeric_cols_with_null = [col_name for col_name, data_type in df.dtypes if data_type in ['int', 'double']]

# Calculate median for each numeric column with null values
median_dict = {}
for col_name in numeric_cols_with_null:
    median_val = df.select(median(col(col_name))).collect()[0][0]
    median_dict[col_name] = median_val

# Replace null values with median for each numeric column
for col_name, median_val in median_dict.items():
    df = df.withColumn(col_name, when(col(col_name).isNull(), median_val).otherwise(col(col_name)))

# Show updated DataFrame
df.show()

+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+-------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+-------+--------+--------+--------+--------+
|   SEQN|SDDSRVYR|RIDSTATR|RIAGENDR|RIDAGEYR|RIDAGEMN|RIDRETH1|RIDRETH3|RIDEXMON|RIDEXAGM|DMQMILIZ|DMQADFC|DMDBORN4|DMDCITZN|DMDYRSUS|DMDEDUC3|DMDEDUC2|DMDMARTL|RIDEXPRG|SIALANG|SIAPROXY|SIAINTRP|FIALANG|FIAPROXY|FIAINTRP|MIALANG|MIAPROXY|MIAINTRP|AIALANGA|DMDHHSIZ|DMDFMSIZ|DMDHHSZA|DMDHHSZB|DMDHHSZE|DMDHRGND|DMDHRAGE|DMDHRBR4|DMDHREDU|DMDHRMAR|DMDHSEDU|    WTINT2YR|    WTMEC2YR|SDMVPSU|SDMVSTRA|INDHHIN2|INDFMIN2|INDFMPIR|
+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+---

In [30]:
from pyspark.sql.functions import col, sum as spark_sum
# Calculate the count of null values in each column
null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
# Convert the result to a dictionary format for easier analysis
null_counts_dict = null_counts.first().asDict()
# Print the count of null values in each column
for column, count in null_counts_dict.items():
    print(f"Column '{column}': {count} null values")

Column 'SEQN': 0 null values
Column 'SDDSRVYR': 0 null values
Column 'RIDSTATR': 0 null values
Column 'RIAGENDR': 0 null values
Column 'RIDAGEYR': 0 null values
Column 'RIDAGEMN': 0 null values
Column 'RIDRETH1': 0 null values
Column 'RIDRETH3': 0 null values
Column 'RIDEXMON': 0 null values
Column 'RIDEXAGM': 0 null values
Column 'DMQMILIZ': 0 null values
Column 'DMQADFC': 0 null values
Column 'DMDBORN4': 0 null values
Column 'DMDCITZN': 0 null values
Column 'DMDYRSUS': 0 null values
Column 'DMDEDUC3': 0 null values
Column 'DMDEDUC2': 0 null values
Column 'DMDMARTL': 0 null values
Column 'RIDEXPRG': 0 null values
Column 'SIALANG': 0 null values
Column 'SIAPROXY': 0 null values
Column 'SIAINTRP': 0 null values
Column 'FIALANG': 0 null values
Column 'FIAPROXY': 0 null values
Column 'FIAINTRP': 0 null values
Column 'MIALANG': 0 null values
Column 'MIAPROXY': 0 null values
Column 'MIAINTRP': 0 null values
Column 'AIALANGA': 0 null values
Column 'DMDHHSIZ': 0 null values
Column 'DMDFMSIZ':

In [5]:
df.printSchema()

root
 |-- SEQN: integer (nullable = true)
 |-- SDDSRVYR: integer (nullable = true)
 |-- RIDSTATR: integer (nullable = true)
 |-- RIAGENDR: integer (nullable = true)
 |-- RIDAGEYR: integer (nullable = true)
 |-- RIDAGEMN: integer (nullable = true)
 |-- RIDRETH1: integer (nullable = true)
 |-- RIDRETH3: integer (nullable = true)
 |-- RIDEXMON: integer (nullable = true)
 |-- RIDEXAGM: integer (nullable = true)
 |-- DMQMILIZ: integer (nullable = true)
 |-- DMQADFC: integer (nullable = true)
 |-- DMDBORN4: integer (nullable = true)
 |-- DMDCITZN: integer (nullable = true)
 |-- DMDYRSUS: integer (nullable = true)
 |-- DMDEDUC3: integer (nullable = true)
 |-- DMDEDUC2: integer (nullable = true)
 |-- DMDMARTL: integer (nullable = true)
 |-- RIDEXPRG: integer (nullable = true)
 |-- SIALANG: integer (nullable = true)
 |-- SIAPROXY: integer (nullable = true)
 |-- SIAINTRP: integer (nullable = true)
 |-- FIALANG: integer (nullable = true)
 |-- FIAPROXY: integer (nullable = true)
 |-- FIAINTRP: int

In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, median

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Replace Null with Median") \
    .getOrCreate()

# Assuming df is your Spark DataFrame with numeric columns containing null values

# Identify numeric columns with null values
numeric_cols_with_null = [col_name for col_name, data_type in df.dtypes 
                          if data_type in [ 'double' ] and df.select(col(col_name)).filter(col(col_name).isNull()).count() > 0]

# Calculate median for each numeric column with null values
median_values = df.select([median(col_name).alias(col_name) for col_name in numeric_cols_with_null]).collect()[0].asDict()

# Replace null values with median for each numeric column
for col_name in numeric_cols_with_null:
    df = df.withColumn(col_name, when(col(col_name).isNull(), median_values[col_name]).otherwise(col(col_name)))

# Show updated DataFrame
df.show()


+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+-------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+-------+--------+--------+--------+--------+
|   SEQN|SDDSRVYR|RIDSTATR|RIAGENDR|RIDAGEYR|RIDAGEMN|RIDRETH1|RIDRETH3|RIDEXMON|RIDEXAGM|DMQMILIZ|DMQADFC|DMDBORN4|DMDCITZN|DMDYRSUS|DMDEDUC3|DMDEDUC2|DMDMARTL|RIDEXPRG|SIALANG|SIAPROXY|SIAINTRP|FIALANG|FIAPROXY|FIAINTRP|MIALANG|MIAPROXY|MIAINTRP|AIALANGA|DMDHHSIZ|DMDFMSIZ|DMDHHSZA|DMDHHSZB|DMDHHSZE|DMDHRGND|DMDHRAGE|DMDHRBR4|DMDHREDU|DMDHRMAR|DMDHSEDU|    WTINT2YR|    WTMEC2YR|SDMVPSU|SDMVSTRA|INDHHIN2|INDFMIN2|INDFMPIR|
+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+---